In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName('LeetCode').getOrCreate()

24/12/24 15:21:41 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/24 15:21:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 15:21:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/24 15:22:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
employees_Spark = spark.read.option('header', True).option('delimiter', ',').csv('Employees.csv')
employeeUNI_Spark = spark.read.option('header', True).option('delimiter', ',').csv('EmployeeUNI.csv')

employees_Pandas = pd.read_csv('Employees.csv')
employeeUNI_Pandas = pd.read_csv('EmployeeUNI.csv')

employees_Spark.show()
employeeUNI_Spark.show()

print(employees_Pandas.head())
print(employeeUNI_Pandas.head())

+---+--------+
| id|    name|
+---+--------+
|  1|   Alice|
|  7|     Bob|
| 11|    Meir|
| 90| Winston|
|  3|Jonathan|
+---+--------+

+---+---------+
| id|unique_id|
+---+---------+
|  3|        1|
| 11|        2|
| 90|        3|
+---+---------+

   id      name
0   1     Alice
1   7       Bob
2  11      Meir
3  90   Winston
4   3  Jonathan
   id  unique_id
0   3          1
1  11          2
2  90          3


In [9]:
def replace_employee_id(employees: pyspark.sql.dataframe.DataFrame, 
                        employee_uni: pyspark.sql.dataframe.DataFrame) \
            -> pyspark.sql.dataframe.DataFrame:
    employees.createOrReplaceTempView('Employees')
    employee_uni.createOrReplaceTempView('EmployeeUNI')
    
    sqlQuery = \
    '''
    SELECT e2.unique_id, e1.name FROM
    Employees e1 LEFT JOIN EmployeeUNI e2
    ON e1.id = e2.id
    '''
    
    output = spark.sql(sqlQuery = sqlQuery)
    return output

output = replace_employee_id(employees_Spark, employeeUNI_Spark)
output.show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     null|   Alice|
|     null|     Bob|
|        2|    Meir|
|        3| Winston|
|        1|Jonathan|
+---------+--------+



In [13]:
def replace_employee_id(employees: pyspark.sql.dataframe.DataFrame, 
                        employee_uni: pyspark.sql.dataframe.DataFrame) \
            -> pyspark.sql.dataframe.DataFrame:
    output = employees.join(employee_uni, on = 'id', how = 'left')\
                        .select(['unique_id', 'name'])
    return output

output = replace_employee_id(employees_Spark, employeeUNI_Spark)
output.show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     null|   Alice|
|     null|     Bob|
|        2|    Meir|
|        3| Winston|
|        1|Jonathan|
+---------+--------+



In [16]:
def replace_employee_id(employees: pd.DataFrame, 
                        employee_uni: pd.DataFrame) -> pd.DataFrame:
    output = employees.merge(employee_uni, on = 'id', how = 'left')
    output = output[['unique_id', 'name']]
    return output

output = replace_employee_id(employees_Pandas, employeeUNI_Pandas)
output.head()

,unique_id,name
0,NaN,Alice
1,NaN,Bob
2,2.0,Meir
3,3.0,Winston
4,1.0,Jonathan
